In [49]:
import pandas as pd
import numpy as np
from sodapy import Socrata

In [51]:
client = Socrata("data.sfgov.org", None)
# results = client.get("cuks-n6tp", limit = 2191368)
results = client.get("cuks-n6tp", limit = 3000000)
results_df = pd.DataFrame.from_records(results)

In [57]:
print(type(results))
print(results[0])
# print(results)

<type 'list'>
{u'category': u'OTHER OFFENSES', u'dayofweek': u'Saturday', u'incidntnum': u'130648192', u'time': u'19:17', u'descript': u'VIOLATION OF RESTRAINING ORDER', u'pddistrict': u'MISSION', u'y': u'37.7651107322703', u'location': {u'type': u'Point', u'coordinates': [-122.432198022433, 37.76511073227]}, u'address': u'2200 Block of MARKET ST', u'date': u'2013-08-03T00:00:00.000', u'x': u'-122.432198022433', u'resolution': u'NONE', u'pdid': u'13064819215161'}


In [58]:
print(type(results_df))
print(results_df.shape)

<class 'pandas.core.frame.DataFrame'>
(2191390, 13)
